In this tutorial, we will learn how to use multiple GPUs using DataParallel.

It’s very easy to use GPUs with PyTorch. You can put the model on a GPU:

```
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
```

Then, you can copy all your tensors to the GPU:

```
mytensor = my_tensor.to(device)
```

Please note that just calling *my_tensor.to(device)* returns a new copy of *my_tensor* on GPU instead of rewriting *my_tensor*. You need to assign it to a new tensor and use that tensor on the GPU.

It’s natural to execute your forward, backward propagations on multiple GPUs. However, Pytorch will only use one GPU by default. You can easily run your operations on multiple GPUs by making your model run parallelly using *DataParallel*:

```
model = nn.DataParallel(model)
```

That’s the core behind this tutorial. We will explore it in more detail below.

# Imports and parameters

Import PyTorch modules and define parameters.

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dummy DataSet

Make a dummy (random) dataset. You just need to implement the getitem

In [2]:
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

# Simple Model

For the demo, our model just gets an input, performs a linear operation, and gives an output. However, you can use DataParallel on any model (CNN, RNN, Capsule Net etc.)

We’ve placed a print statement inside the model to monitor the size of input and output tensors. Please pay attention to what is printed at batch rank 0.

In [3]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

# Create Model and DataParallel

This is the core part of the tutorial. First, we need to make a model instance and check if we have multiple GPUs. If we have multiple GPUs, we can wrap our model using *nn.DataParallel*. Then we can put our model on GPUs by *model.to(device)*

In [4]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model.to(device)

Let's use 3 GPUs!


DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

# Run the Model

Now we can see the sizes of input and output tensors.

In [5]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

RuntimeError: CUDA error: out of memory (malloc at /opt/conda/conda-bld/pytorch_1556653183467/work/c10/cuda/CUDACachingAllocator.cpp:241)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x45 (0x7f05100d9dc5 in /home/anhaoran/anaconda3/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x16901 (0x7f05087a4901 in /home/anhaoran/anaconda3/lib/python3.6/site-packages/torch/lib/libc10_cuda.so)
frame #2: <unknown function> + 0x17347 (0x7f05087a5347 in /home/anhaoran/anaconda3/lib/python3.6/site-packages/torch/lib/libc10_cuda.so)
frame #3: at::native::empty_cuda(c10::ArrayRef<long>, c10::TensorOptions const&) + 0x274 (0x7f0515c86f34 in /home/anhaoran/anaconda3/lib/python3.6/site-packages/torch/lib/libcaffe2_gpu.so)
frame #4: at::CUDAType::empty(c10::ArrayRef<long>, c10::TensorOptions const&) const + 0x19b (0x7f05146844cb in /home/anhaoran/anaconda3/lib/python3.6/site-packages/torch/lib/libcaffe2_gpu.so)
frame #5: torch::autograd::VariableTyp

# Summary

DataParallel splits your data automatically and sends job orders to multiple models on several GPUs. After each model finishes their job, DataParallel collects and merges the results before returning it to you.

For more information, please check out https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html.